In [ ]:
# Loading Packages
import pandas as pd, json
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

# Reading in Data
df1 = pd.read_csv('bettinglines20.csv')
df1['Year'] = 2020
df2 = pd.read_csv('bettinglines21.csv')
df2['Year'] = 2021
df3 = pd.read_csv('bettinglines22.csv')
df3['Year'] = 2022
df4 = pd.read_csv('bettinglines23.csv')
df4['Year'] = 2023
df5 = pd.read_csv('bettinglines24.csv')
df5['Year'] = 2024
df6 = pd.read_csv('bettinglines25.csv')
df6['Year'] = 2025

# Combining Data Sets
data = pd.concat([df1, df2, df3, df4, df5, df6], ignore_index = True)
# Removing Game Not Played Yet
data = data.drop([242,243,244])

# Making Spread Easier to Understand
# Now Formatted as:
    # (+) BYU Favored to Win
    # (-) BYU Favored to Lose
data[['spread_team', 'spread_value']] = data['FormattedSpread'].str.rsplit(' ', n=1, expand=True)
data['spread_value'] = data['spread_value'].astype(float)
def normalize_spread(row):
    if row['spread_team'] == 'BYU':
        return f"{-row['spread_value']}"  
    else:
        return f"{(row['spread_value'])}"  
    
data['NormalizedSpread'] = data.apply(normalize_spread, axis=1)
data['NormalizedSpread'] = data['NormalizedSpread'].astype(float)

# Calculating Difference in Scores
data['BYU_Diff'] = np.where(
    data['HomeTeam'] == 'BYU', 
    data['HomeScore'] - data['AwayScore'],  
    np.where(
        data['AwayTeam'] == 'BYU',
        data['AwayScore'] - data['HomeScore'],  
        np.nan 
    )
)

# Creating Interesting Columns
data['TotalScore'] = data['HomeScore'] + data['AwayScore']
data['Home'] = (data['HomeTeam'] == 'BYU')
data['Cover'] = data['BYU_Diff'] > data['NormalizedSpread']
data['Over/Under'] = data['TotalScore'] - data['OverUnder']
data['Win'] = data['BYU_Diff'] > 0
data['Favored'] = data['NormalizedSpread'] > 0
data['Year'] = data['Year'].astype('category')

# Dropping Unnecessary Columns
data = data.drop('spread_team', axis = 1)
data = data.drop('spread_value', axis = 1)
data = data.drop('FormattedSpread', axis = 1)
data = data.drop('Spread', axis = 1)
data = data.drop('Id', axis = 1)
data = data.drop('OpeningSpread', axis = 1)
data = data.drop('OpeningOverUnder', axis = 1)
data = data.drop('HomeMoneyline', axis = 1)
data = data.drop('AwayMoneyline', axis = 1)

# Saving CSV
data.to_csv('BettingLinesFinal.csv', index = False)

LineProvider
Bovada                       64
consensus                    37
teamrankings                 36
William Hill (New Jersey)    35
DraftKings                   29
ESPN Bet                     24
numberfire                   14
Caesars                       9
SugarHouse                    3
Name: count, dtype: int64